Teniendo el Dataset preparado compenzamos a hacer las primeras pruebas con modlos de regresion y clasificacion 

Comenzaremos preparando los datos en Train y Test

In [10]:
#Cargamos librerias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

from sklearn.metrics import mean_squared_error, r2_score

#para guardar el modelo

from pickle import dump 

#eliminamos mensajes de error
import warnings
warnings.filterwarnings('ignore')

In [15]:
#cargamos el dataset
metadata_processed = pd.read_csv ('/Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/metadata_processed.csv')
metadata_processed

,target,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,token_solar,token_special,token_spitz,token_spreading,token_squamous,token_sunexposed,token_superficial,token_trichilemmal,token_type,token_verruca
0,0,60.0,3.04,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,...,0,0,0,0,0,0,0,0,0,0
1,0,60.0,1.10,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,...,0,0,0,0,0,0,0,0,0,0
2,0,60.0,3.40,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,...,0,0,0,0,0,0,0,0,0,0
3,0,65.0,3.22,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,...,0,0,0,0,0,0,0,0,0,0
4,0,55.0,2.73,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381909,0,70.0,6.80,22.574335,14.944666,27.663259,26.767135,35.705132,30.656525,50.784168,...,0,0,0,0,0,0,0,0,0,0
381910,0,60.0,3.11,19.977640,16.026870,34.158840,31.011870,39.571870,34.908400,59.678970,...,0,0,0,0,0,0,0,0,0,0
381911,0,65.0,2.05,17.332567,12.364397,29.845326,26.500073,34.513206,29.242644,59.854275,...,0,0,0,0,0,0,0,0,0,0
381912,0,30.0,2.80,22.288570,9.564721,28.431200,27.012250,36.126360,28.655640,51.905420,...,0,0,0,0,0,0,0,0,0,0


para evitar problemas en la creacion y entrenamiento de los modelos, haremos una revision de caracteres especiales en el nombre de nuestras variables 

In [16]:
# Limpiar los nombres de las columnas eliminando o reemplazando caracteres especiales y espacios en blanco
metadata_processed.columns = metadata_processed.columns.str.replace('[\[\]\{\}\:\,\s]', '_', regex=True)

Tomando en cuenta que ya tenemos el dataset preparado para las primeras pruebas, separamos los conjuntos de train y test

In [17]:
# Separar características y variable objetivo
X = metadata_processed.drop('target', axis=1)
y = metadata_processed['target']

In [18]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape},{y_train.shape}')
print(f'Tamaño del conjunto de prueba: {X_test.shape},{y_test.shape}')

Tamaño del conjunto de entrenamiento: (305531, 136),(305531,)
Tamaño del conjunto de prueba: (76383, 136),(76383,)


ya que tenemos menos de un 1% de valores positivos en el target, comprobaremos si en el conjunto de test hay algun valor para hacer la comprobacion, asi nos aseguramos que se pueda hacer las pruebas con valores nunca vistos para los modelo

In [19]:

# Comprobamos cuántos valores de 'y_test' son iguales a 1
target_1_count = np.sum(y_test == 1)

# Mostramos el resultado
print(f"El número de observaciones en el conjunto de test donde la variable objetivo es 1: {target_1_count}")


El número de observaciones en el conjunto de test donde la variable objetivo es 1: 77


Creamos los modelos y eliminamos mensajes de error

In [20]:
#eliminamos mensajes de error
import warnings
#warnings.filterwarnings('ignore')

# modelos
ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.01, random_state=42)
gb =GradientBoostingClassifier (n_estimators=100, learning_rate=0.01, random_state=42)
xgb = XGBClassifier(n_estimators=100, learning_rate=0.01, random_state = 42, use_label_encoder=False, eval_metric='mlogloss')
lgb = LGBMClassifier(n_estimators=100, learning_rate=0.01, random_state = 42, verbose=-1)

#modelos con valores por defecto
'''ada = AdaBoostClassifier()
gb = GradientBoostingClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()'''

# entrenamiento
ada.fit(X_train, y_train)
gb.fit(X_train, y_train)
xgb.fit(X_train, y_train)
lgb.fit(X_train, y_train)

#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)
#warnings.filterwarnings("ignore", category=UserWarning, module='xgboost')

LGBMClassifier(learning_rate=0.01, random_state=42, verbose=-1)

In [21]:
# hacemos la prediccion
ada_y_pred_test = ada.predict(X_test)
gb_y_pred_test = gb.predict(X_test)
xgb_y_pred_test = xgb.predict(X_test)
lgb_y_pred_test = lgb.predict(X_test)

ada_y_pred_train = ada.predict(X_train)
gb_y_pred_train = gb.predict(X_train)
xgb_y_pred_train = xgb.predict(X_train)
lgb_y_pred_train = lgb.predict(X_train)

revisamos las metricas para comprobar la eficacia de los modelos, con valores sin parametrar

In [22]:
#revisamos metricas
ada_accuracy_test = accuracy_score(y_test, ada_y_pred_test)
ada_accuracy_train = accuracy_score(y_train, ada_y_pred_train)

gb_accuracy_test = accuracy_score(y_test, gb_y_pred_test)
gb_accuracy_train = accuracy_score(y_train, gb_y_pred_train)

xgb_accuracy_test = accuracy_score(y_test, xgb_y_pred_test)
xgb_accuracy_train = accuracy_score(y_train, xgb_y_pred_train)

lgb_accuracy_test = accuracy_score(y_test, lgb_y_pred_test)
lgb_accuracy_train = accuracy_score(y_train, lgb_y_pred_train)

print('AdaBoost')
print("Accuracy Test: ", ada_accuracy_test)
print("Accuracy Train: ", ada_accuracy_train)

print('Gradient Boosting')
print("Accuracy Test: ", gb_accuracy_test)
print("Accuracy Train: ", gb_accuracy_train)

print('XGBoost')
print("Accuracy Test: ", xgb_accuracy_test)
print("Accuracy Train: ", xgb_accuracy_train)

print('LightGBM')
print("Accuracy Test: ", lgb_accuracy_test)
print("Accuracy Train: ", lgb_accuracy_train)

AdaBoost
Accuracy Test:  1.0
Accuracy Train:  1.0
Gradient Boosting
Accuracy Test:  1.0
Accuracy Train:  1.0
XGBoost
Accuracy Test:  1.0
Accuracy Train:  1.0
LightGBM
Accuracy Test:  1.0
Accuracy Train:  1.0
